## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kropotkin,RU,45.4375,40.5756,35.01,88,100,10.38,snow
1,1,Albany,US,42.6001,-73.9662,66.99,18,74,3.00,broken clouds
2,2,New Norfolk,AU,-42.7826,147.0587,64.00,79,100,1.01,overcast clouds
3,3,Vezirkopru,TR,41.1436,35.4547,37.40,75,75,12.66,light rain
4,4,Tasiilaq,GL,65.6145,-37.6368,12.20,67,0,2.30,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Eldorado,BR,-23.7869,-54.2836,81.97,73,93,3.83,overcast clouds
8,8,Itacoatiara,BR,-3.1431,-58.4442,82.22,81,54,2.48,broken clouds
9,9,Buin,CL,-33.7333,-70.7500,78.80,36,0,9.22,clear sky
16,16,Hambantota,LK,6.1241,81.1185,78.67,79,94,10.96,overcast clouds
17,17,Atuona,PF,-9.8000,-139.0333,80.91,69,52,19.24,broken clouds
18,18,Kavieng,PG,-2.5744,150.7967,80.65,81,100,13.49,moderate rain
20,20,Mount Isa,AU,-20.7333,139.5000,75.20,83,90,4.61,overcast clouds
21,21,Souillac,MU,-20.5167,57.5167,78.80,94,40,7.07,scattered clouds
24,24,Ambon,ID,-3.6954,128.1814,75.20,94,20,6.91,few clouds
27,27,Rikitea,PF,-23.1203,-134.9692,79.41,75,14,10.96,few clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Eldorado,BR,81.97,overcast clouds,-23.7869,-54.2836,
8,Itacoatiara,BR,82.22,broken clouds,-3.1431,-58.4442,
9,Buin,CL,78.80,clear sky,-33.7333,-70.7500,
16,Hambantota,LK,78.67,overcast clouds,6.1241,81.1185,
17,Atuona,PF,80.91,broken clouds,-9.8000,-139.0333,
18,Kavieng,PG,80.65,moderate rain,-2.5744,150.7967,
20,Mount Isa,AU,75.20,overcast clouds,-20.7333,139.5000,
21,Souillac,MU,78.80,scattered clouds,-20.5167,57.5167,
24,Ambon,ID,75.20,few clouds,-3.6954,128.1814,
27,Rikitea,PF,79.41,few clouds,-23.1203,-134.9692,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Eldorado,BR,81.97,overcast clouds,-23.7869,-54.2836,Villa Verde Hotel
8,Itacoatiara,BR,82.22,broken clouds,-3.1431,-58.4442,BBB Rooms Centro Itacoatiara AM
9,Buin,CL,78.80,clear sky,-33.7333,-70.7500,Paihuen
16,Hambantota,LK,78.67,overcast clouds,6.1241,81.1185,Bungalow 63
17,Atuona,PF,80.91,broken clouds,-9.8000,-139.0333,Villa Enata
18,Kavieng,PG,80.65,moderate rain,-2.5744,150.7967,Nusa Island Retreat
20,Mount Isa,AU,75.20,overcast clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
21,Souillac,MU,78.80,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
24,Ambon,ID,75.20,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
27,Rikitea,PF,79.41,few clouds,-23.1203,-134.9692,Pension Maro'i


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))